In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import Util

In [2]:
df = pd.read_csv('hungary_chickenpox.csv',sep=',')
df = df.drop(columns = ['Date'])

In [3]:
df

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,168,79,30,173,169,42,136,120,162,36,130,57,2,178,66,64,11,29,87,68
1,157,60,30,92,200,53,51,70,84,28,80,50,29,141,48,29,58,53,68,26
2,96,44,31,86,93,30,93,84,191,51,64,46,4,157,33,33,24,18,62,44
3,163,49,43,126,46,39,52,114,107,42,63,54,14,107,66,50,25,21,43,31
4,122,78,53,87,103,34,95,131,172,40,61,49,11,124,63,56,7,47,85,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,95,12,41,6,39,0,16,15,14,10,56,7,13,122,4,23,4,11,110,10
518,43,39,31,10,34,3,2,30,25,19,34,20,18,70,36,5,23,22,63,9
519,35,7,15,0,0,0,7,7,4,2,30,36,4,72,5,21,14,0,17,10
520,30,23,8,0,11,4,1,9,10,17,27,17,21,12,5,17,1,1,83,2


In [4]:
num_data = 400

In [5]:
df_1 = df.iloc[:num_data]

In [6]:
data = df_1.to_numpy()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
input_size = len(df.columns)
output_size= len(df.columns)
hidden_size = len(df.columns)
num_layers = 3
z_dim = 20
batch_size = 20

In [8]:
# Convert numpy array to PyTorch tensor
tensor = torch.tensor(data, dtype=torch.float32)

# Create TimeSeriesDataset object
dataset = TensorDataset(tensor)

# Create DataLoader for batch processing
dataloader = DataLoader(dataset, batch_size, shuffle=True)

In [9]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.tensor(sample, dtype=torch.float)

In [10]:
# class Generator(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, num_layers):
#         super().__init__()
#         self.rnn = nn.GRU(input_size, hidden_size, num_layers)
#         self.fc = nn.Linear(hidden_size, output_size)
#         self.dropout = nn.Dropout(0.3)
#         self.relu = nn.ReLU()
        
#     def forward(self, x):
#         x, _ = self.rnn(x)
#         x = self.dropout(x)
#         x = self.fc(x)
#         x = self.relu(x)
#         return x

In [11]:
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super().__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        return x

In [12]:
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x.squeeze()

In [13]:
# def train_gan(generator, discriminator, dataloader, batch_size, num_epochs, lr_g=0.001, lr_d=0.001, critic_iters=3):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     generator.to(device)
#     discriminator.to(device)
#     optimizer_g = optim.Adam(generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
#     optimizer_d = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))

#     for epoch in range(num_epochs):
#         for i, real_samples in enumerate(dataloader):
#             real_samples = torch.stack(real_samples, dim=0).to(device)

#             # Train discriminator
#             for critic_iter in range(critic_iters):
#                 noise = torch.randn(batch_size, 1, 20, device=device)
#                 fake_samples = generator(noise)
#                 fake_output = discriminator(fake_samples.detach())
#                 real_output = discriminator(real_samples)

#                 # Wasserstein loss
#                 loss_d = -torch.mean(real_output) + torch.mean(fake_output)
#                 gradient_penalty = calc_gradient_penalty(discriminator, real_samples, fake_samples.detach(), device=device)
#                 loss_d += gradient_penalty

#                 optimizer_d.zero_grad()
#                 loss_d.backward()
#                 optimizer_d.step()

#             # Train generator
#             noise = torch.randn(batch_size, 1, 20, device=device)
#             fake_samples = generator(noise)
#             fake_output = discriminator(fake_samples)
#             loss_g = -torch.mean(fake_output)

#             optimizer_g.zero_grad()
#             loss_g.backward()
#             optimizer_g.step()

#             if i % 100 == 0:
#                 print(f"Epoch {epoch+1}/{num_epochs} | Discriminator Loss: {loss_d.item():.4f} | Generator Loss: {loss_g.item():.4f}")

# def calc_gradient_penalty(discriminator, real_samples, fake_samples, device):
#     alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=device)
#     interpolated = (alpha * real_samples + (1 - alpha) * fake_samples).requires_grad_(True)
#     prob_interpolated = discriminator(interpolated)
#     gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated,
#                                     grad_outputs=torch.ones(prob_interpolated.size(), device=device),
#                                     create_graph=True, retain_graph=True)[0]
#     gradients = gradients.view(real_samples.size(0), -1)
#     gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
#     return gradient_penalty

In [19]:
def train_gan(generator, discriminator, dataloader,batch_size, num_epochs, lr_g=0.001, lr_d=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    generator.to(device)
    discriminator.to(device)
    criterion = nn.BCELoss()
    optimizer_g = optim.Adam(generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))
    
    for epoch in range(num_epochs):
        for i, real_samples in enumerate(dataloader):
            
            
            # Train discriminator
            optimizer_d.zero_grad()
            
            # Train on real samples
            real_samples = torch.stack(real_samples, dim=0).to(device)
            real_labels = torch.ones(batch_size, device=device)
            real_output = discriminator(real_samples)
            loss_d_real = criterion(real_output, real_labels)
            
            # Train on fake samples
            noise = torch.randn(batch_size, 1, 20, device=device)
            fake_samples = generator(noise)
            fake_labels = torch.zeros(batch_size, device=device)
            fake_output = discriminator(fake_samples.detach())
            loss_d_fake = criterion(fake_output, fake_labels)
            
            loss_d = loss_d_real + loss_d_fake
            loss_d.backward()
            optimizer_d.step()
            
            # Train generator
            optimizer_g.zero_grad()
            
            noise = torch.randn(batch_size, 1, 20, device=device)
            fake_samples = generator(noise)
            fake_labels = torch.ones(batch_size, device=device)
            fake_output = discriminator(fake_samples)
            loss_g = criterion(fake_output, fake_labels)
            
            loss_g.backward()
            optimizer_g.step()
            
            if i % 100 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Discriminator Loss: {loss_d.item():.4f} | Generator Loss: {loss_g.item():.4f}")
                

In [15]:
generator = Generator(input_size,output_size, hidden_size, num_layers)
discriminator = Discriminator(input_size, hidden_size, num_layers)

In [20]:
train_gan(generator, discriminator, dataloader,batch_size, num_epochs = 100, lr_g=0.001, lr_d=0.001)

Epoch 1/100 | Discriminator Loss: 0.8319 | Generator Loss: 0.6931
Epoch 2/100 | Discriminator Loss: 0.8665 | Generator Loss: 0.6931
Epoch 3/100 | Discriminator Loss: 0.9011 | Generator Loss: 0.6931
Epoch 4/100 | Discriminator Loss: 0.9704 | Generator Loss: 0.6931
Epoch 5/100 | Discriminator Loss: 0.9014 | Generator Loss: 0.6931
Epoch 6/100 | Discriminator Loss: 0.9012 | Generator Loss: 0.6931
Epoch 7/100 | Discriminator Loss: 0.8318 | Generator Loss: 0.6931
Epoch 8/100 | Discriminator Loss: 0.7626 | Generator Loss: 0.6931
Epoch 9/100 | Discriminator Loss: 0.9012 | Generator Loss: 0.6931
Epoch 10/100 | Discriminator Loss: 0.7972 | Generator Loss: 0.6931
Epoch 11/100 | Discriminator Loss: 0.9011 | Generator Loss: 0.6931
Epoch 12/100 | Discriminator Loss: 0.9011 | Generator Loss: 0.6931
Epoch 13/100 | Discriminator Loss: 0.9705 | Generator Loss: 0.6931
Epoch 14/100 | Discriminator Loss: 0.9012 | Generator Loss: 0.6931
Epoch 15/100 | Discriminator Loss: 0.9705 | Generator Loss: 0.6931
Epoc

In [17]:
z = torch.randn(size=(100, 20), device=device)
with torch.no_grad():
    fake_samples = generator(z)

In [18]:
fake_samples

tensor([[ 0.0000,  0.0000, 26.8818,  ...,  0.0000, 20.6079,  0.0000],
        [36.1054,  0.0000, 30.2158,  ...,  0.0000, 10.6867,  0.0000],
        [70.1318,  0.0000, 53.2137,  ...,  0.0000, 38.7306,  0.0000],
        ...,
        [61.8561,  0.0000,  0.0000,  ...,  6.0459, 32.2684,  0.0000],
        [14.7510,  0.0000,  0.0000,  ...,  0.0000, 15.4998,  0.0000],
        [50.4327,  0.0000,  0.0000,  ...,  0.0000, 37.0186,  0.0000]],
       device='cuda:0')